In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy

from scipy import linalg
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import spsolve
from scipy.sparse import diags
from scipy.sparse import identity

In [2]:
#GiBUU FinalEvents.dat data struct
# 1:Run  2:Event  3:ID 4:Charge     5:perweight   6:position(1)   7:position(2)   8:position(3)   9:momentum(0)  
#10:momentum(1)  11:momentum(2)  12:momentum(3)     13:history  14:production_ID  15:enu

# sscanf(tmpline.Data(),"%d %d %d %d %lf %lf %lf %lf %lf %lf %lf %lf %*d %d %lf", 
#& 1-tmprun, & 2-tmpevent, & 3-tmpid, & 4-tmpcharge, & 5-tmppw, & 6-tmppos1, & 7-tmppos2, & 8-tmppos3, 
#& 9-tmptote, & 10-tmpmom1, & 11-tmpmom2, & 12-tmpmom3, /*&history,*/ & 14-tmpprod, & 15-tmpenu);

    #1: nucleon (QE)
    #2-31: non-strange baryon resonance (as in IdTable)
    #32: pi neutron-background (e.g. nu + n -> mu + pi+ + n)
    #33: pi proton-background (e.g. nu + n -> mu + pi0 + p)
    #34: DIS
    #35: 2p2h QE
    #36: 2p2h Delta
    #37: two pion background


In [3]:
data_1_line = np.zeros(15)
data_1_line = [1, 4, 1, 1, 0.000000E+00, 0.000000E+00, 0.000000E+00, 0.000000E+00, 9.276043E-01, 1.171171E-01,-6.729271E-03, -1.334868E-01, 0, 35, 1.926118E+00]

run_n = data_1_line[0] #run
event_n = data_1_line[1] #event
particle_id = data_1_line[2] #id of particle
perweight = data_1_line[4] #weight of event
prod_id = data_1_line[13] #production id
beam_E = data_1_line[14] #energy of incoming neutrino from beam

part_pos = np.array([data_1_line[5], data_1_line[6], data_1_line[7]]) #position of particle in final state
part_4vec = np.array([data_1_line[8], data_1_line[9], data_1_line[10], data_1_line[11]])



In [4]:
part_4vec

array([ 0.9276043 ,  0.1171171 , -0.00672927, -0.1334868 ])

In [5]:
PionMass = 139.570/1e3


def read_event_csv(filename, run_n, event_n): #return event with the run number and event nunber
    filein = open(filename, "r")            # Open file for reading
    inputd = filein.readlines()
    inputdata = []
    temp_data_1_line = []

    leng = len(inputd)
    print("the number of line of the txt file list is : " + str(leng))

    for i in range(1,leng):
        tokens = inputd[i].split(",")
        
        for j in range(0,15):
            if j<4:
                tokens[j] = int(tokens[j])
            elif j<12:
                tokens[j] = float(tokens[j])
            elif j<14:
                tokens[j] = int(tokens[j])
            else:
                tokens[j] = float(tokens[j])
                
        temp_data_1_line = tokens

        if (temp_data_1_line[0] == float(run_n)) and (temp_data_1_line[1] == float(event_n)):
            inputdata.append(temp_data_1_line)
            
    del(inputd)            
    return inputdata

def read_event(filename, run_n, event_n): #return event with the run number and event nunber
    filein = open(filename, "r")            # Open file for reading
    inputd = filein.readlines()
    inputdata = []
    temp_data_1_line = []

    leng = len(inputd)
    print("the number of line of the txt file list is : " + str(leng))

    for i in range(1,leng):
        tokens = inputd[i].split()#("	")
        
        for j in range(0,15):
            if j<4:
                tokens[j] = int(tokens[j])
            elif j<12:
                tokens[j] = float(tokens[j])
            elif j<14:
                tokens[j] = int(tokens[j])
            else:
                tokens[j] = float(tokens[j])
                
        temp_data_1_line = tokens

        if (temp_data_1_line[0] == float(run_n)) and (temp_data_1_line[1] == float(event_n)):
            inputdata.append(temp_data_1_line)
         
    del(inputd)
    return inputdata

#4-vector is in form of [t,x,y,z]

def get_P(Lorentz_4vec):#return momentum from 4 momentum
    return np.linalg.norm(np.array([Lorentz_4vec[1],Lorentz_4vec[2],Lorentz_4vec[3]]))

def get_mag2(Lorentz_4vec):#return magnitude sq from 4 momentum
    return Lorentz_4vec[0]**2-(Lorentz_4vec[1]**2+Lorentz_4vec[2]**2+Lorentz_4vec[3]**2)

def get_mag(Lorentz_4vec):#return magnitude sq from 4 momentum
    return np.sqrt(Lorentz_4vec[0]**2-(Lorentz_4vec[1]**2+Lorentz_4vec[2]**2+Lorentz_4vec[3]**2))

def Dot_Lor(Lorentz_4vec1, Lorentz_4vec2):#return magnitude sq from 4 momentum
    return Lorentz_4vec1[0]*Lorentz_4vec2[0]-Lorentz_4vec1[1]*Lorentz_4vec2[1]-Lorentz_4vec1[2]*Lorentz_4vec2[2]-Lorentz_4vec1[3]*Lorentz_4vec2[3]

def get_theta(Lorentz_4vec):#return angle in degrees from z axis (0,0,1) from 4 momentum
    z_hat = np.array([0,0,1])
    p_vec = np.array([Lorentz_4vec[1],Lorentz_4vec[2],Lorentz_4vec[3]])
    p_vec = p_vec / np.linalg.norm(p_vec)
    angle_rad = np.arccos(np.dot(z_hat, p_vec))
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

#4-vector is in form of [t,x,y,z] instead of [x,y,z,t] for TLorentz vec for ROOT
def get_anal_event(event_lst):
    line_n = len(event_lst)
    print("num of FS particles in event : "+str(line_n))
    run_n, event_n, particle_id, perweight, prod_id, beam_E, part_posm, part_4vec, part_pos = [], [], [], [], [], [], [], [], []
    for i in range(0,line_n):
        run_n.append(event_lst[i][0]) #run
        event_n.append(event_lst[i][1]) #event
        particle_id.append(event_lst[i][2]) #id of particle
        perweight.append(event_lst[i][4]) #weight of event
        prod_id.append(event_lst[i][13]) #production id
        beam_E.append(event_lst[i][14]) #energy of incoming neutrino from beam

        part_pos.append(np.array([event_lst[i][5], event_lst[i][6], event_lst[i][7]])) #position of particle in final state
        part_4vec.append(np.array([event_lst[i][8], event_lst[i][9], event_lst[i][10], event_lst[i][11]]))
        if (particle_id[i] == 902) or (particle_id[i] == 901): #particle is muon get scatter should be changed to 901 for electron
            PU4pScatter = part_4vec[i]
            
        if perweight[i]<1e-10: #initial nucleon which should have 0 perweight
            iniNfullp = part_4vec[i]
    
    beamE = sum(beam_E)/len(beam_E)
    print("beam E : "+str(beamE))
    beamMass = PionMass
    pz = np.sqrt(beamE*beamE-beamMass*beamMass)
    beamp = np.array([beamE,0,0,pz])
    dummyNu = np.array([beamE,0,0,beamE])
    
    print(" PU4pScatter : "+str(PU4pScatter))
    lvq= dummyNu - PU4pScatter
    
    scattermomentum = get_P(PU4pScatter)
    scattertheta = get_theta(PU4pScatter)
    
    Q2 = -get_mag2(lvq)
    print(" iniNfullp : "+str(iniNfullp))
    dummyW = lvq+iniNfullp
    print(" lvq : "+str(lvq))
    print(" dummyW : "+str(dummyW))
    Wtrue = get_mag(dummyW)

    xBj = Q2/(2*Dot_Lor(iniNfullp,lvq))
    #prod id
    #1: nucleon (QE)
    #2-31: non-strange baryon resonance (as in IdTable)
    #32: pi neutron-background (e.g. nu + n -> mu + pi+ + n)
    #33: pi proton-background (e.g. nu + n -> mu + pi0 + p)
    #34: DIS
    #35: 2p2h QE
    #36: 2p2h Delta
    #37: two pion background
    print()
    if prod_id[0] == 1:
        print("prod id : "+str(prod_id[0])+" event mode : QE")
    elif prod_id[0] <= 33:
        print("prod id : "+str(prod_id[0])+" event mode : RES")
    elif prod_id[0] == 34:
        print("prod id : "+str(prod_id[0])+" event mode : DIS")
    elif prod_id[0] == 35:
        print("prod id : "+str(prod_id[0])+" event mode : 2p2h QE")
        
    
    print("run : "+str(run_n[0])+"\n event : "+str(event_n[0])+"\n prod_id : "+str(prod_id[0])+"\n beamE : "+str(beamE)+"\n scatter p : "+str(scattermomentum))
    print("scattertheta : "+str(scattertheta)+"\n Q2 : "+str(Q2)+"\n Wtrue : "+str(Wtrue)+"\n xBj : "+str(xBj))
    
    return [run_n[0], event_n[0], prod_id[0], beamE, scattermomentum, scattertheta, Q2, Wtrue, xBj]
    
    

In [6]:
filename = "FinalEvents.dat"
event = read_event(filename,2,192)
print(event)

the number of line of the txt file list is : 10167723
[[2, 192, 902, -1, 0.0001895717, 0.0, 0.0, 0.0, 3.172954, -0.782087, -0.1741759, 3.068302, 0, 33, 5.165583], [2, 192, 1, 0, 0.0, 0.0, 0.0, 0.0, 0.9259209, 0.1856034, -0.0002121793, 0.1020846, 0, 33, 5.165583], [2, 192, 1, 1, 0.0001895717, -5.131554, 6.308086, 18.10065, 1.223003, -0.2427319, 0.2598311, 0.6996157, 1001001, 33, 5.165583], [2, 192, 101, 0, 0.0001895717, 30.76335, -3.550623, 6.204753, 0.6039399, 0.5693755, -0.06407947, 0.1319274, 0, 33, 5.165583], [2, 192, 101, -1, 0.0001895717, 19.57923, -5.322415, 21.03987, 0.4808295, 0.2940201, -0.0901691, 0.3428916, 2000007, 33, 5.165583], [2, 192, 101, 1, 0.0001895717, -1.51025, 18.82256, 8.49741, 0.2473333, -0.0502453, 0.1916702, 0.05355113, 3000002, 33, 5.165583], [2, 192, 1, 1, 0.0001895717, 8.389218, -3.214107, 18.7153, 1.268924, 0.2528522, -0.1533509, 0.8017933, 3000002, 33, 5.165583]]


In [17]:
event = read_event(filename,20,1892)
print(event)
get_anal_event(event)

the number of line of the txt file list is : 10167723
num of FS particles in event : 4
beam E : 3.442237
 PU4pScatter : [ 2.128097   0.5021488 -0.2423581  2.051034 ]
 iniNfullp : [ 0.9158984   0.01526664 -0.04621209  0.01148205]
 lvq : [ 1.31414   -0.5021488  0.2423581  1.391203 ]
 dummyW : [ 2.2300384  -0.48688216  0.19614601  1.40268505]

prod id : 33 event mode : RES
run : 20 event : 1892 prod_id : 33 beamE : 3.442237 scatter p : 2.1254720264292004
scattertheta : 15.208383672321517 Q2 : 0.5193727135860493 Wtrue : 1.65227673862942 xBj : 0.21523747970082271


[20,
 1892,
 33,
 3.442237,
 2.1254720264292004,
 15.208383672321517,
 0.5193727135860493,
 1.65227673862942,
 0.21523747970082271]

In [18]:
filename = "nu_mu_sample_jobcard4_MINERvA\GiBUUMINERvALE_nu_T0_Carbon_1_out1/FinalEvents.dat"
event = read_event(filename,2,12)
print(event)

the number of line of the txt file list is : 13669809
[[2, 12, 902, -1, 3.801258e-05, 0.0, 0.0, 0.0, 1.831453, -0.05037215, 0.2958759, 1.803601, 0, 35, 1.955155], [2, 12, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.9158672, -0.1977203, 0.03576699, 0.03877834, 0, 35, 1.955155], [2, 12, 1, 1, 3.801258e-05, -1.192684, -0.3328881, 10.83487, 0.9791979, -0.02407384, 0.03010662, 0.2783857, 0, 35, 1.955155], [2, 12, 1, 1, 3.801258e-05, -2.558745, -0.8691897, -5.964559, 0.962879, -0.01214603, 0.06179227, -0.2081531, 0, 35, 1.955155]]


In [33]:
event = read_event(filename,7,8248)
print(event)
get_anal_event(event)

the number of line of the txt file list is : 13669809
[[7, 8248, 902, -1, 0.0001112534, 0.0, 0.0, 0.0, 2.529119, -0.1568405, -0.1737898, 2.516044, 0, 34, 5.151251], [7, 8248, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.9085629, 0.06306799, -0.08375778, 0.01982305, 0, 34, 5.151251], [7, 8248, 1, 0, 0.0001112534, -6.993914, 1.597815, 15.53209, 1.14994, -0.2589687, 0.1286147, 0.5990922, 1001001, 34, 5.151251], [7, 8248, 101, 0, 0.0001112534, 19.66363, -12.18493, 13.18073, 0.3718823, 0.2789817, -0.1487294, 0.1389289, 2000010, 34, 5.151251], [7, 8248, 1, 1, 0.0001112534, 1.25765, 3.363306, 29.00245, 2.671542, 0.3086289, 0.3639089, 2.455527, 2000010, 34, 5.151251], [7, 8248, 1, 1, 0.0001112534, -5.092485, -3.863809, -9.417087, 0.9974436, -0.08951902, -0.1062925, -0.3094157, 2000002, 34, 5.151251], [7, 8248, 1, 1, 0.0001112534, 10.94556, -7.168531, 0.6983392, 1.069495, 0.4610607, -0.2202834, 0.05360076, 4000002, 34, 5.151251], [7, 8248, 1, 0, 0.0001112534, -1.378054, 9.203741, -6.309528, 1.110349, -0.0411863

[7,
 8248,
 34,
 5.151251000000001,
 2.5269110089119264,
 5.315613939720586,
 0.12354154444929133,
 2.3152011579738923,
 0.02645657359410647]

In [34]:
filename = "nu_e_sample_jobcard13_E7\GiBUUMINERvALE_nu_T0_Carbon_1_out1/FinalEvents.dat"
event = read_event(filename,2,12)
print(event)

the number of line of the txt file list is : 2520474
[[2, 12, 901, -1, 3.084846e-05, 0.0, 0.0, 0.0, 1.595742, -0.1367419, -0.2170863, 1.574981, 0, 35, 1.631925], [2, 12, 1, 0, 0.0, 0.0, 0.0, 0.0, 0.9097171, 0.06701041, -0.09384776, -0.1111457, 0, 35, 1.631925], [2, 12, 1, 1, 3.084846e-05, -2.579089, 0.586928, 0.02537319, 0.939303, -0.09090551, -0.1850331, 0.1323261, 0, 35, 1.631925]]


In [43]:
event = read_event(filename,2,11494)
print(event)
get_anal_event(event)

the number of line of the txt file list is : 2520474
[[2, 11494, 901, -1, 9.870628e-05, 0.0, 0.0, 0.0, 2.451184, -0.104074, 0.1793049, 2.442401, 0, 33, 3.85099], [2, 11494, 1, 1, 0.0, 0.0, 0.0, 0.0, 0.9183071, -0.06572914, 0.1151882, -0.1233749, 0, 33, 3.85099], [2, 11494, 1, 1, 9.870628e-05, 7.27128, -15.75614, 8.35934, 1.201066, 0.1894457, -0.6319405, 0.3570395, 0, 33, 3.85099], [2, 11494, 1, 1, 9.870628e-05, 2.215473, -4.322532, 0.44121, 0.9446193, -0.05081354, -0.1022997, -0.009800713, 4001001, 33, 3.85099], [2, 11494, 1, 1, 9.870628e-05, -0.3145993, 20.42611, 17.2977, 1.58053, 0.05141028, 0.9962378, 0.7893651, 1101001, 33, 3.85099], [2, 11494, 1, 1, 9.870628e-05, 5.255197, -3.568153, -0.8856054, 1.048029, 0.4381197, -0.1310421, -0.09696247, 6002001, 33, 3.85099], [2, 11494, 1, 0, 9.870628e-05, -0.7912965, -8.120287, -3.934768, 0.9963142, -0.01090502, -0.2665864, -0.2039869, 4001001, 33, 3.85099], [2, 11494, 1, 1, 9.870628e-05, -4.413761, 1.903372, 9.655876, 1.028291, -0.1349313, 0

[2,
 11494,
 33,
 3.85099,
 2.4511842626455094,
 4.8518498631406874,
 0.06764777792501042,
 1.9277686017895117,
 0.022750513097931677]